In [189]:
# download the latest NHC advisories (TEXT rather than a-decks since they are going to be updated faster)
# parse them to recreate the forecast intensities
# use the CSV tables from nhc_errors to produce probabilities (RI or not)
# ONLY AL AND EP FOR NOW

from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
import pytz
import pandas as pd
# to create word numbers to numbers mapping
import inflect
import time
import numpy as np

# FORECAST ADVISORY TEXT BASIN NAMES (THESE ARE THE ABBREVS USED IN FOLDER NAMES) ('AT' WILL BECOME 'AL' in df):
basin_initials = ['AT', 'EP', 'CP']
# currently 5 bins to put active storms (may increase to 10 in future)
num_bins = 5

nhc_text_url_base = 'https://www.nhc.noaa.gov/archive/text'

# AL AND EP csv (tables) for percentiles (created from nhc_errors.ipynb)
nhc_error_tables = ['table1.csv',
 'table2.csv',
 'table3.csv',
 'table4.csv',
 'table5.csv',
 'table6.csv',
 'table7.csv',
 'table8.csv',
 'table9.csv',
 'table10.csv']

# when calculating RI adjusted probabilities, limit calculations with RI up to tau (low samples)
max_tau_by_basin = {
    'AL': 36,
    'EP': 72
}

# Based on "INITIAL THOUGHTS" sections in nhc_errors.ipynb, that conditionalizes on whether RI is in forecast
#   each RI, NO_RI is a list of tuples:
#      the first value is the table # (as it appears in nhc_errors), and
#      the second value is the weight for the corresponding probability
adjust_RI_weights = {
    'AL': {
        'RI': [(4, 0.0275), (1, 0.9725)],
        'NO_RI': [(3, 0.6679), (2, 0.3321)]
    },
    'EP': {
        'RI': [(9, 0.0184), (6, 0.9816)],
        'NO_RI': [(8, 0.6225), (7, 0.3375)]
    }
}

In [190]:
# Saffir-Simpson (in kt)
saffir_categories_kt = {
    'TD': (0, 33),
    'TS': (34, 63),
    'CAT1': (64, 82),
    'CAT2': (83, 95),
    'CAT3': (96, 112),
    'CAT4': (113, 136),
    'CAT5': (137, np.inf)
}

# NHC advisories only have increments of 5kt, so use categories that fit that
advisory_categories_kt = {
    'TD': (0, 30),
    'TS': (35, 60),
    'CAT1': (65, 80),
    'CAT2': (85, 95),
    'CAT3': (100, 110),
    'CAT4': (115, 135),
    'CAT5': (140, np.inf)
}

#######################################################################
# the choice of which of the above to use does alter the probabilities
#######################################################################
categories_kt = advisory_categories_kt

p = inflect.engine()
word_names = []
word_names_to_numbers = {}
for i in range(1, 100):
    word = p.number_to_words(i).lower()
    word_names.append(word)
    word_names_to_numbers[word] = i

month_abbrevs = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_abbrevs_to_month_num = {v: i for i, v in enumerate(month_abbrevs, 1)}

# initialize min, max names & percentiles for each category
# initialize probs (based on percentiles)
cat_min_max_kt = {}
p_cat_labels = {}
for cat, [cat_min, cat_max] in categories_kt.items():
    cat_min_max_kt[f'{cat}_MIN'] = cat_min
    cat_min_max_kt[f'{cat}_MAX'] = cat_max
    p_cat_labels[cat] = f'P({cat})'

In [191]:
def get_text_file_names(base_url):
    response = requests.get(base_url)
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = []
    for link in soup.find_all('a'):
        href = link.get('href').rstrip("/")
        if href and not href.startswith('../') and href.endswith('.txt'):
            urls.append(href)
    return urls

def get_text_file(url):
    response = requests.get(url)
    time.sleep(1)
    if response.status_code == 200:
        return response.text
    else:
        return None

# get urls of text folders for all basins
def get_nhc_text_folder_urls(year):
    year_str = str(year)
    #https://www.nhc.noaa.gov/archive/text/TCMAT1
    nhc_text_folders = []
    for basin in basin_initials:
        basin = basin.upper()
        for bin_num in range(1, num_bins+1):
            bin_num_str = str(bin_num)
            url = f'{nhc_text_url_base}/TCM{basin}{bin_num_str}/{year_str}'
            nhc_text_folders.append(url)
    return nhc_text_folders

# get for all basins
def get_last_forecasts_text():
    year = datetime.now().year
    nhc_text_folder_urls = get_nhc_text_folder_urls(year)

    if not nhc_text_folder_urls:
        return None
    
    txts = []
    for folder_url in nhc_text_folder_urls:
        flist = get_text_file_names(folder_url)
        if flist and len(flist) > 0:
            last_txt_file_name = sorted(flist, reverse=True)[0]
            url = f'{folder_url}/{last_txt_file_name}'
            txt = get_text_file(url)
            if txt:
                txts.append(txt)
    return txts

def df_from_ofcl_forecast(txt):
    #print(txt)
    df_ofcl_forecast = pd.DataFrame({
                        'ATCF_ID': pd.Series(dtype='str'),
                        'BASIN': pd.Series(dtype='str'),
                        'STORM_NUM': pd.Series(dtype='int'),
                        'SEASON': pd.Series(dtype='str'),
                        'NAME': pd.Series(dtype='str'),
                        'INIT_TIME_UTC': pd.Series(dtype='datetime64[ns, UTC]'),
                        'VALID_TIME_UTC': pd.Series(dtype='datetime64[ns, UTC]'),
                        'TAU': pd.Series(dtype='int'),
                        'VMAX_KT': pd.Series(dtype='float'),
                        'STORM_TYPE': pd.Series(dtype='str')
    })

    lines = txt.split('\n')
    r_name = re.compile('(?P<init_storm_type>.*?) +(?P<storm_name>\w+)(?:-[EACLPW])? +FORECAST/ADVISORY NUMBER +(?P<advisory_num>\d+)')
    r_atcf = re.compile('^NWS (?:NATIONAL HURRICANE CENTER MIAMI FL|CENTRAL PACIFIC HURRICANE CENTER HONOLULU HI) +(?P<atcf_id>\w+)')
    r_advisory_date = re.compile(f'^(?P<advisory_time>\d\d\d\d) UTC \w+ +(?P<month_abbrev>\w+) +(?P<advisory_day_num>\d+) +(?P<advisory_year>\d\d\d\d)')
    r_init_time = re.compile('^AT (?P<init_day_num>\d+)/(?P<init_time_hhmm>\d\d\d\d)Z CENTER')
    r_init_vmax = re.compile('^MAX SUSTAINED WINDS +(?P<init_vmax>\d+) +KT')
    r_valid_time_with_center = re.compile('^(?:FORECAST|OUTLOOK) VALID (?P<valid_day_num>\d+)/(?P<valid_time_hhmm>\d\d\d\d)Z +[\w\.]+[NS] +[\w\.]+[WE]\.*(?P<valid_storm_type>.*)')
    r_valid_time_without_center = re.compile('^(?:FORECAST|OUTLOOK) VALID (?P<valid_day_num>\d+)/(?P<valid_time_hhmm>\d\d\d\d)Z\.*(?P<valid_storm_type>.*)')
    r_valid_vmax = re.compile('^MAX WIND +(?P<valid_vmax>\d+) +KT')
    valid_datetime = None

    ofcl_forecast = {}
    
    for line in lines:
        res_name = re.match(r_name, line)
        res_atcf = re.match(r_atcf, line)
        res_advisory_date = re.match(r_advisory_date, line)
        res_init_time = re.match(r_init_time, line)
        res_init_vmax = re.match(r_init_vmax, line)
        res_valid_time = re.match(r_valid_time_with_center, line)
        if not res_valid_time:
            res_valid_time = re.match(r_valid_time_without_center, line)
        res_valid_vmax = re.match(r_valid_vmax, line)
        if res_name:
            init_storm_type = res_name['init_storm_type'].strip()
            storm_name = res_name['storm_name'].strip()
            advisory_num = int(res_name['advisory_num'].strip())
            if storm_name.lower() in word_names:
                # this is a (special) potential cyclone advisory as it has a number instead of a name
                storm_name = str(word_names_to_numbers[storm_name.lower()])
        elif res_atcf:
            atcf_id = res_atcf['atcf_id'].strip()
            atcf_basin = atcf_id[0:2]
            afcf_storm_num = int(atcf_id[2:4])
            afcf_season = int(atcf_id[4:8])
        elif res_advisory_date:
            advisory_time = res_advisory_date['advisory_time'].strip()
            month_abbrev = res_advisory_date['month_abbrev'].strip().lower()
            advisory_month_num = month_abbrevs_to_month_num[month_abbrev]
            advisory_day_num = int(res_advisory_date['advisory_day_num'].strip())
            advisory_year = int(res_advisory_date['advisory_year'].strip())
            # we need the advisory date/day num for forecasts near midnight at the end/start of a month:
            #   to check if init time are less than, or valid times are more than, the advisory time to adjust the month
            advisory_date_str = f'{advisory_year}-{advisory_month_num:02}-{advisory_day_num:02} {advisory_time}'
            advisory_datetime = datetime.strptime(advisory_date_str, "%Y-%m-%d %H%M").replace(tzinfo=pytz.utc)
        elif res_init_time:
            init_day_num = int(res_init_time['init_day_num'].strip())
            init_time_hhmm = res_init_time['init_time_hhmm'].strip()
            # check to see if same month
            init_day_before_advisory = (init_day_num != (advisory_day_num - 1))
            init_day_after_advisory = (init_day_num != (advisory_day_num + 1))
            if init_day_before_advisory or init_day_after_advisory:
                init_month_num = advisory_month_num
                init_year = advisory_year
            elif init_day_before_advisory:
                # init is from previous month (relative to advisory time)
                init_month_num = advisory_month_num - 1
                if init_month_num == 0:
                    init_month_num = 12
                    init_year = advisory_year - 1
                else:
                    init_year = advisory_year
            elif init_day_after_advisory:
                # init is from previous month (relative to advisory time)
                # this should never happen as usually the advisory time is after init time
                init_month_num = advisory_month_num + 1
                if init_month_num == 13:
                    init_month_num = 1
                    init_year = advisory_year + 1
                else:
                    init_year = advisory_year
            init_date_str = f'{init_year}-{init_month_num:02}-{init_day_num:02} {init_time_hhmm}'
            init_datetime = datetime.strptime(init_date_str, "%Y-%m-%d %H%M").replace(tzinfo=pytz.utc)

            # init vmax comes before init time
            ofcl_forecast[init_datetime] = init_vmax
            tau = 0
            row = [
                    atcf_id,
                    atcf_basin,
                    afcf_storm_num,
                    afcf_season,
                    storm_name,
                    init_datetime,
                    init_datetime,
                    tau,
                    init_vmax,
                    init_storm_type
                ]
            new_df = pd.DataFrame([row], columns=df_ofcl_forecast.columns)
            df_ofcl_forecast = pd.concat([df_ofcl_forecast, new_df])
        elif res_init_vmax:
            init_vmax = int(res_init_vmax['init_vmax'].strip())
        elif res_valid_time:
            valid_day_num = int(res_valid_time['valid_day_num'].strip())
            valid_time_hhmm = res_valid_time['valid_time_hhmm'].strip()
            if 'valid_storm_type' in res_valid_time.groupdict():
                valid_storm_type = res_valid_time['valid_storm_type'].strip()
            else:
                valid_storm_type = init_storm_type

            # check to see if same month
            valid_day_before_advisory = (valid_day_num != (advisory_day_num - 1))
            valid_day_after_advisory = (valid_day_num != (advisory_day_num + 1))
            if valid_day_before_advisory or valid_day_after_advisory:
                valid_month_num = advisory_month_num
                valid_year = advisory_year
            elif valid_day_before_advisory:
                # valid is from previous month (relative to advisory time)
                valid_month_num = advisory_month_num - 1
                if valid_month_num == 0:
                    valid_month_num = 12
                    valid_year = advisory_year - 1
                else:
                    valid_year = advisory_year
            elif valid_day_after_advisory:
                # valid is from previous month (relative to advisory time)
                # this should never happen as usually the advisory time is after valid time
                valid_month_num = advisory_month_num + 1
                if valid_month_num == 13:
                    valid_month_num = 1
                    valid_year = advisory_year + 1
                else:
                    valid_year = advisory_year
            valid_date_str = f'{valid_year}-{valid_month_num:02}-{valid_day_num:02} {valid_time_hhmm}'
            valid_datetime = datetime.strptime(valid_date_str, "%Y-%m-%d %H%M").replace(tzinfo=pytz.utc)

            if "dissipat" in valid_storm_type.lower():
                tau = int((valid_datetime - init_datetime).total_seconds() / 3600)
                valid_vmax = np.nan
                ofcl_forecast[valid_datetime] = valid_vmax
                row = [
                    atcf_id,
                    atcf_basin,
                    afcf_storm_num,
                    afcf_season,
                    storm_name,
                    init_datetime,
                    valid_datetime,
                    tau,
                    valid_vmax,
                    valid_storm_type
                ]
                new_df = pd.DataFrame([row], columns=df_ofcl_forecast.columns)
                df_ofcl_forecast = pd.concat([df_ofcl_forecast, new_df])
                # reset datetime
                valid_datetime = None
        elif res_valid_vmax:
            valid_vmax = int(res_valid_vmax['valid_vmax'].strip())
            # add valid to forecast
            ofcl_forecast[valid_datetime] = valid_vmax
            tau = int((valid_datetime - init_datetime).total_seconds() / 3600)
            row = [
                    atcf_id,
                    atcf_basin,
                    afcf_storm_num,
                    afcf_season,
                    storm_name,
                    init_datetime,
                    valid_datetime,
                    tau,
                    valid_vmax,
                    valid_storm_type
                ]
            new_df = pd.DataFrame([row], columns=df_ofcl_forecast.columns)
            df_ofcl_forecast = pd.concat([df_ofcl_forecast, new_df])
            # reset datetime
            valid_datetime = None
    return df_ofcl_forecast

# get for all basins
def get_last_forecast_dfs():
    # get the text for the last forecast for every bin of all basins
    txts = get_last_forecasts_text()
    df_ofcl_forecasts = None
    for txt in txts:
        try:
            df_ofcl_forecast = df_from_ofcl_forecast(txt)
            
            if type(df_ofcl_forecasts) is type(None) or (
                (type(df_ofcl_forecasts) is type(pd.DataFrame())) and df_ofcl_forecast.empty
            ):
                df_ofcl_forecasts = df_ofcl_forecast
            else:
                if (type(df_ofcl_forecast) is type(pd.DataFrame())) and not (df_ofcl_forecast.empty):
                    df_ofcl_forecasts = pd.concat([df_ofcl_forecasts, df_ofcl_forecast])
        except Exception as e:
            print("Failed to parse forecast:", e)
            traceback.print_exc(limit=None, file=None, chain=True)
            print("")
            print("Text of forecast:")
            print(txt)
    return df_ofcl_forecasts

def get_df_nhc_error_table_from_csv(table_file_path):
    df = pd.read_csv(table_file_path)
    # Filter rows that have '%' in the first column
    df = df[df.iloc[:, 0].str.contains('%')]

    # Rename the first column to 'percentile' and reset the index
    df = df.rename(columns={df.columns[0]: 'percentile'}).reset_index(drop=True)

    # Convert the 'percentile' column to float (remove '%' and divide by 100)
    df['percentile'] = df['percentile'].str.rstrip('%')

    # Convert the values in the DataFrame to integers
    df.iloc[1:] = df.iloc[1:].astype(float)
        
    # Check for columns with NaN values
    columns_with_nan = df.columns[df.isna().any()].tolist()

    # Drop columns with NaN values
    df = df.drop(columns=columns_with_nan)

    df['percentile'] = df['percentile'].astype(float) / 100.0

    # Set the index name to None
    df.index.name = None
    return df

def get_df_with_deltas(df_no_deltas):
    df_ofcl_forecasts_with_deltas = df_no_deltas.copy(deep=True)
    df_ofcl_forecasts_with_deltas = df_ofcl_forecasts_with_deltas.reset_index(drop=True)
    df_ofcl_forecasts_with_deltas['INIT_VMAX_KT'] = np.nan
    df_ofcl_forecasts_with_deltas['DELTA_VMAX_KT'] = np.nan
    df_ofcl_forecasts_with_deltas['RI'] = False

    # SET INIT_MAX_KT FOR EACH ADVISORY
    for i in range(0, len(df_ofcl_forecasts_with_deltas)):
        storm_id = df_ofcl_forecasts_with_deltas.iloc[i]['ATCF_ID']
        init_time = df_ofcl_forecasts_with_deltas.iloc[i]['INIT_TIME_UTC']
        tau = df_ofcl_forecasts_with_deltas.iloc[i]['TAU']
        if tau == 0:
            init_vmax = df_ofcl_forecasts_with_deltas.iloc[i]['VMAX_KT']
        else:
            init_vmax = df_ofcl_forecasts_with_deltas.loc[
                (df_ofcl_forecasts_with_deltas['ATCF_ID'] == storm_id) &
                (df_ofcl_forecasts_with_deltas['INIT_TIME_UTC'] == init_time) &
                (df_ofcl_forecasts_with_deltas['TAU'] == 0), 'VMAX_KT'
            ].values[0]
        df_ofcl_forecasts_with_deltas.at[i, 'INIT_VMAX_KT'] = init_vmax

    # CALCULATE DELTA_VMAX_KT

    df_ofcl_forecasts_with_deltas['DELTA_VMAX_KT'] = df_ofcl_forecasts_with_deltas.apply(
        lambda x: x['VMAX_KT'] - x['INIT_VMAX_KT'], axis=1
    )

    # CALCULATE RI (for tau = 12h to 72h)
    df_ofcl_forecasts_with_deltas['RI'] = df_ofcl_forecasts_with_deltas.apply(
        lambda x: is_rapid_intensification(x['TAU'], x['DELTA_VMAX_KT']), axis=1
    )
    return df_ofcl_forecasts_with_deltas

# calculate VMAX_ERROR: valid vmax - init vmax (from more recent advisory)
def get_df_with_vmax_error(df_with_deltas):
    df_ofcl_forecasts_errors = df_with_deltas.copy(deep=True)
    df_ofcl_forecasts_errors['VMAX_ERROR'] = np.nan
    # SET INIT_MAX_KT FOR EACH ADVISORY
    for i in range(0, len(df_ofcl_forecasts_errors)):
        storm_id = df_ofcl_forecasts_errors.iloc[i]['ATCF_ID']
        valid_time = df_ofcl_forecasts_errors.iloc[i]['VALID_TIME_UTC']
        valid_vmax = df_ofcl_forecasts_errors.iloc[i]['VMAX_KT']
        init_vmax_row = df_ofcl_forecasts_errors.loc[
                (df_ofcl_forecasts_errors['ATCF_ID'] == storm_id) &
                (df_ofcl_forecasts_errors['INIT_TIME_UTC'] == valid_time) &
                (df_ofcl_forecasts_errors['TAU'] == 0), 'VMAX_KT'
        ]
        if not init_vmax_row.empty:
            init_vmax = init_vmax_row.values[0]
            vmax_error = valid_vmax - init_vmax
            df_ofcl_forecasts_errors.at[i, 'VMAX_ERROR'] = vmax_error
    return df_ofcl_forecasts_errors
        
def get_df_nhc_error_tables():
    df_error_tables = {}
    for table_file_path in nhc_error_tables:
        table_num = int(re.findall(r'table(\d+).csv', table_file_path)[0])
        df_error_table = get_df_nhc_error_table_from_csv(table_file_path)
        df_error_tables[table_num] = df_error_table
    return df_error_tables

def get_table_basin_offset(basin):
    table_offset = 0
    if basin == 'AT' or basin == 'AL':
        table_offset = 0
    elif basin == 'EP':
        table_offset = 5
    return table_offset

# table_num is indexed 1 (1-10), while the list for dfs is indexed 0 (0-4)
# also, table_num is the full set (1-10), while the dfs list index is modulo 5, since it includes both basins
def get_dfs_index_from_table_num(table_num):
    return ((table_num - 1) % 5)

# takes a list of intensity errors and calculates the percentile for a given tau
# returns a dict, with a tuple of the range of probabilities for each given intensity error
#    (if <0.01 or >.99, returns a tuple of (0,0) or (1,1) respectively)
# returns None if tau is not in table
def get_percentiles_for_intensity_errors(tau, basin, intensity_errors, basin_table_num=5):
    # TODO: Haven't done CP tables yet
    if basin == 'CP':
        return None
    table_offset = get_table_basin_offset(basin)
    df_base_prob = df_error_tables[basin_table_num + table_offset]
    tau_str = str(tau)
    if tau_str in df_base_prob.columns:
        prob_tuples_by_intensity_errors = {}
        for intensity_error in intensity_errors:
            prob_tuple = find_percentile_range(df_base_prob, intensity_error, tau_str)
            prob_tuples_by_intensity_errors[intensity_error] = prob_tuple
        return prob_tuples_by_intensity_errors    
    else:
        return None
    
# returns a tuple (if outside the range return (0,0.1) or (0.99,1.0)
# this uses 'intensity' var shorthand for forecast (valid) intensity error from init (valid intensity - init intensity)
def find_percentile_range(df, intensity, tau):
    # Find the corresponding 'percentile' values
    probabilities = df['percentile']
    values = df[tau]

    # Check if there are values equal to the given intensity
    equal_intensity = probabilities[values == intensity]

    if not equal_intensity.empty:
        # Values equal to the intensity are found
        return (min(equal_intensity), max(equal_intensity))
    else:
        
        # Check if the column is increasing or decreasing
        increasing = values.is_monotonic_increasing
        decreasing = values.is_monotonic_decreasing
        
        # Values equal to the intensity are not found
        min_prob = probabilities[values <= intensity]
        max_prob = probabilities[values >= intensity]

        if min_prob.empty:
            # Intensity is greater than the maximum value in the column
            if increasing:
                return (0.0, 0.01)
            elif decreasing:
                return (0, 0.1)
            else:
                # should never happen
                return None
        elif max_prob.empty:
            # Intensity is less than the minimum value in the column
            if increasing:
                return (0.99, 1.0)
            elif decreasing:
                return (0, 0.1)
            else:
                # should never happen
                return None
        else:
            # Intensity is between the minimum and maximum values in the column
            return (max(min_prob), min(max_prob))

# calculate probs from percentiles and returns dfs for each table
def get_dfs_with_probs(df_ofcl_forecasts):
    df_ofcl_forecasts_copy = df_ofcl_forecasts.copy(deep=True)
    # calculate the deltas
    df_ofcl_forecasts_with_deltas = get_df_with_deltas(df_ofcl_forecasts_copy)
    # calculate the vmax errors using older forecasts
    df_ofcl_forecasts_errors = get_df_with_vmax_error(df_ofcl_forecasts_with_deltas)
    
    # calculate probs for each basin table (1-5 is for AL 1-5 and EP 6-10)
    dfs_with_probs = []
    for basin_table in range(1,6):
        df_with_probs = df_ofcl_forecasts_errors.copy(deep=True)

        df_ofcl_forecasts_min_max_cat_percentiles = df_with_probs.copy(deep=True)
        for cat_name in cat_min_max_kt.keys():
            df_ofcl_forecasts_min_max_cat_percentiles[f'{cat_name}_L'] = np.nan
            df_ofcl_forecasts_min_max_cat_percentiles[f'{cat_name}_U'] = np.nan

        df_ofcl_forecasts_min_max_cat_percentiles = df_with_probs.copy(deep=True)
        for p_cat_label in p_cat_labels.values():
            df_ofcl_forecasts_min_max_cat_percentiles[p_cat_label] = np.nan

        for i in range(0, len(df_ofcl_forecasts_min_max_cat_percentiles)):
            storm_id = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i]['ATCF_ID']
            storm_basin = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i]['BASIN']
            valid_time = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i]['VALID_TIME_UTC']
            tau = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i]['TAU']
            valid_vmax = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i]['VMAX_KT']

            if np.isnan(valid_vmax):
                continue

            vmax_errors = []
            for cat_name, cat_vmax in cat_min_max_kt.items():
                vmax_error = valid_vmax - cat_vmax
                vmax_errors.append(vmax_error)
            percentiles = get_percentiles_for_intensity_errors(tau, storm_basin, vmax_errors)
            percentiles_values = list(percentiles.values())
            if percentiles:
                for x, cat_name in enumerate(cat_min_max_kt.keys()):
                    percentile_L, percentile_U = percentiles_values[x]
                    df_ofcl_forecasts_min_max_cat_percentiles.at[i, f'{cat_name}_L'] = percentile_L
                    df_ofcl_forecasts_min_max_cat_percentiles.at[i, f'{cat_name}_U'] = percentile_U
            for cat, [cat_min, cat_max] in categories_kt.items():
                cat_min_L_col = f'{cat}_MIN_L'
                cat_min_U_col = f'{cat}_MIN_U'
                cat_max_L_col = f'{cat}_MAX_L'
                cat_max_U_col = f'{cat}_MAX_U'

                cat_min_L = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i][cat_min_L_col]
                cat_min_U = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i][cat_min_U_col]
                cat_max_L = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i][cat_max_L_col]
                cat_max_U = df_ofcl_forecasts_min_max_cat_percentiles.iloc[i][cat_max_U_col]
                min_percentile = min(cat_min_L, cat_min_U, cat_max_L, cat_max_U)
                max_percentile = max(cat_min_L, cat_min_U, cat_max_L, cat_max_U)
                prob = max_percentile - min_percentile
                p_cat_label = p_cat_labels[cat]
                df_with_probs.at[i, p_cat_label] = prob


        dfs_with_probs.append(df_with_probs)

    return dfs_with_probs

def get_df_probs_ri_adjusted(dfs):
    df_probs_ri = dfs[4].copy(deep=True)

    for i in range(0, len(df_probs_ri)):
        ri = df_probs_ri.iloc[i]['RI']
        delta_vmax = df_probs_ri.iloc[i]['DELTA_VMAX_KT']
        tau = df_probs_ri.iloc[i]['TAU']
        if np.isnan(delta_vmax):
            # don't calculate on rows where there are no probabilities
            continue
        
        storm_id = df_probs_ri.iloc[i]['ATCF_ID']
        storm_basin = df_probs_ri.iloc[i]['BASIN']
        valid_time = df_probs_ri.iloc[i]['VALID_TIME_UTC']

        if tau > max_tau_by_basin[storm_basin]:
            # too few samples
            continue

        if ri:
            ri_key = 'RI'
        else:
            ri_key = 'NO_RI'
        
        ri_weights = adjust_RI_weights[storm_basin][ri_key]
        p_cat_column_names = list(p_cat_labels.values())
        for p_cat_column_name in p_cat_column_names:
            p = 0
            for table_num, table_weight in ri_weights:
                dfs_index = get_dfs_index_from_table_num(table_num)
                table_p = dfs[dfs_index].iloc[i][p_cat_column_name]
                ri_adj_p = table_weight * table_p
                p += ri_adj_p
            if not np.isnan(p):
                prev_p = df_probs_ri.iloc[i][p_cat_column_name]
                df_probs_ri.at[i, p_cat_column_name] = p
                delta_p = p - prev_p
                if (np.abs(delta_p) > 0.01):
                    print(f"{storm_id}")
                    print(f"  New adjust p: {p:.4f}")
                    print(f"    prev p: {prev_p:.4f}")
                    print(f"    delta p: {delta_p:.4f}")
    
    return df_probs_ri
    
def is_rapid_intensification(valid_h, intensity_change):
    ri = False
    if np.isnan(intensity_change):
        return ri
    if valid_h == 0:
        # don't do RI for base time (000h)
        return ri
    # https://journals.ametsoc.org/view/journals/wefo/35/6/WAF-D-19-0253.1.xml#bib15
    # 'RI is therefore defined as an increase of at least 20 kt in 12 h, 30 kt in 24 h, 45 kt in 36 h, and 55 kt in 48 h'
    rapid_intensification_threshold = np.NaN
    if valid_h <= 12:
        rapid_intensification_threshold = 20
    elif valid_h <= 24:
        rapid_intensification_threshold = 30
    elif valid_h <= 36:
        rapid_intensification_threshold = 45
    elif valid_h <= 48:
        rapid_intensification_threshold = 55
    elif valid_h <= 60:
        rapid_intensification_threshold = 60
    elif valid_h <= 72:
        rapid_intensification_threshold = 65
    # only consider rapid intensification for the above periods
    if np.isnan(rapid_intensification_threshold):
        return ri
    if (intensity_change >= rapid_intensification_threshold):
        ri = True
    return ri



In [192]:
df_error_tables = get_df_nhc_error_tables()

In [193]:
df_ofcl_forecasts = get_last_forecast_dfs()
pd.set_option('display.max_rows', 500)

In [204]:
dfs_with_probs = get_dfs_with_probs(df_ofcl_forecasts)

In [205]:
df_ri_adj = get_df_probs_ri_adjusted(dfs_with_probs)

EP162023
  New adjust p: 0.9504
    prev p: 0.9900
    delta p: -0.0396
EP172023
  New adjust p: 0.7584
    prev p: 0.7900
    delta p: -0.0316
EP172023
  New adjust p: 0.6624
    prev p: 0.6900
    delta p: -0.0276
EP172023
  New adjust p: 0.4320
    prev p: 0.4500
    delta p: -0.0180
EP172023
  New adjust p: 0.2688
    prev p: 0.2800
    delta p: -0.0112
EP172023
  New adjust p: 0.4224
    prev p: 0.4400
    delta p: -0.0176
EP172023
  New adjust p: 0.2976
    prev p: 0.3100
    delta p: -0.0124
EP172023
  New adjust p: 0.3936
    prev p: 0.4100
    delta p: -0.0164
EP172023
  New adjust p: 0.5664
    prev p: 0.5900
    delta p: -0.0236
EP172023
  New adjust p: 0.4800
    prev p: 0.5000
    delta p: -0.0200
EP132023
  New adjust p: 0.7680
    prev p: 0.8000
    delta p: -0.0320
EP132023
  New adjust p: 0.8064
    prev p: 0.8400
    delta p: -0.0336
EP132023
  New adjust p: 0.7584
    prev p: 0.7900
    delta p: -0.0316
EP142023
  New adjust p: 0.9216
    prev p: 0.9600
    delta p: 

In [206]:
#get_percentiles_for_intensity_errors(12, 'AT', [-6, -5, 0])

In [207]:
# probabilities won't be normalized by category...
# also <0.01 and 0.01 show as 0.01, just as 0.99 and >0.99 show as 0.99

dfs_to_check = {
    "NO RI ADJ.": dfs_with_probs[4],
    "RI ADJ.": df_ri_adj
}

#df_check.loc[(df_check['NAME'] == 'NORMA')]
for k, df_check in dfs_to_check.items():
    print("=======================================================================")
    print(f"{k} PROBABILITIES (CALCULATED USING PERCENTILES FROM 1981-2022)")
    print("=======================================================================")
    print("Norma a hurricane?")
    for idx in df_check.loc[(df_check['NAME'] == 'NORMA')].index:
        cat1 = df_check.iloc[idx]['P(CAT1)']
        cat2 = df_check.iloc[idx]['P(CAT2)']
        cat3 = df_check.iloc[idx]['P(CAT3)']
        cat4 = df_check.iloc[idx]['P(CAT4)']
        cat5 = df_check.iloc[idx]['P(CAT5)']
        ts = df_check.iloc[idx]['P(TS)']
        td = df_check.iloc[idx]['P(TD)']
        
        hurricane = cat1+cat2+cat3+cat4+cat5
        not_hurricane = ts+td
        
        norm = hurricane / (hurricane + not_hurricane)
        
        dt = df_check.iloc[idx]['VALID_TIME_UTC']
        dt_str = dt.strftime("%Y-%m-%d %HZ")
        print(f"{dt_str} : {norm*100: 5.1f} %")

    print("")

    print("Norma a major hurricane?")
    for idx in df_check.loc[(df_check['NAME'] == 'NORMA')].index:
        cat1 = df_check.iloc[idx]['P(CAT1)']
        cat2 = df_check.iloc[idx]['P(CAT2)']
        cat3 = df_check.iloc[idx]['P(CAT3)']
        cat4 = df_check.iloc[idx]['P(CAT4)']
        cat5 = df_check.iloc[idx]['P(CAT5)']
        ts = df_check.iloc[idx]['P(TS)']
        td = df_check.iloc[idx]['P(TD)']
        
        major_hurricane = cat3+cat4+cat5
        not_major_hurricane = ts+td+cat1+cat2
        
        norm = major_hurricane / (major_hurricane + not_major_hurricane)
        
        dt = df_check.iloc[idx]['VALID_TIME_UTC']
        dt_str = dt.strftime("%Y-%m-%d %HZ")
        print(f"{dt_str} : {norm*100: 5.1f} %")

    print("")

    print("Tammy a hurricane?")
    for idx in df_check.loc[(df_check['NAME'] == 'TAMMY')].index:
        cat1 = df_check.iloc[idx]['P(CAT1)']
        cat2 = df_check.iloc[idx]['P(CAT2)']
        cat3 = df_check.iloc[idx]['P(CAT3)']
        cat4 = df_check.iloc[idx]['P(CAT4)']
        cat5 = df_check.iloc[idx]['P(CAT5)']
        ts = df_check.iloc[idx]['P(TS)']
        td = df_check.iloc[idx]['P(TD)']
        
        hurricane = cat1+cat2+cat3+cat4+cat5
        not_hurricane = ts+td
        
        norm = hurricane / (hurricane + not_hurricane)
        
        dt = df_check.iloc[idx]['VALID_TIME_UTC']
        dt_str = dt.strftime("%Y-%m-%d %HZ")
        print(f"{dt_str} : {norm*100: 5.1f} %")

    print("")
    print("")

NO RI ADJ. PROBABILITIES (CALCULATED USING PERCENTILES FROM 1981-2022)
Norma a hurricane?
2023-10-19 18Z :  98.1 %
2023-10-20 06Z :  98.0 %
2023-10-20 18Z :  97.9 %
2023-10-21 06Z :  84.5 %
2023-10-21 18Z :  65.6 %
2023-10-22 06Z :  31.6 %
2023-10-22 18Z :  24.7 %
2023-10-23 18Z :   9.5 %
2023-10-24 18Z :   nan %

Norma a major hurricane?
2023-10-19 18Z :  96.1 %
2023-10-20 06Z :  84.2 %
2023-10-20 18Z :  38.1 %
2023-10-21 06Z :  14.4 %
2023-10-21 18Z :   9.4 %
2023-10-22 06Z :   4.1 %
2023-10-22 18Z :   4.1 %
2023-10-23 18Z :   3.2 %
2023-10-24 18Z :   nan %

Tammy a hurricane?
2023-10-19 18Z :   4.8 %
2023-10-20 06Z :   9.8 %
2023-10-20 18Z :  25.0 %
2023-10-21 06Z :  41.4 %
2023-10-21 18Z :  54.6 %
2023-10-22 06Z :  70.4 %
2023-10-22 18Z :  74.0 %
2023-10-23 18Z :  75.8 %
2023-10-24 18Z :  75.5 %


RI ADJ. PROBABILITIES (CALCULATED USING PERCENTILES FROM 1981-2022)
Norma a hurricane?
2023-10-19 18Z :  98.1 %
2023-10-20 06Z :  98.0 %
2023-10-20 18Z :  97.9 %
2023-10-21 06Z :  84.5 %


In [208]:
# over a 24 hour period
np.average([0.94,
0.9693877551020408,
0.9789473684210526,
0.9270833333333333])

0.9538546142141067

In [209]:
np.average([
0.9805825242718447,
0.9800000000000001,
0.9795918367346939,
0.96875
])

0.9772310902516346

In [210]:
dfs_with_probs[4][dfs_with_probs[4]['ATCF_ID'] == 'EP172023']

,ATCF_ID,BASIN,STORM_NUM,SEASON,NAME,INIT_TIME_UTC,VALID_TIME_UTC,TAU,VMAX_KT,STORM_TYPE,...,DELTA_VMAX_KT,RI,VMAX_ERROR,P(TD),P(TS),P(CAT1),P(CAT2),P(CAT3),P(CAT4),P(CAT5)
29,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-19 18:00:00+00:00,0,110.0,HURRICANE,...,0.0,False,0.0,0.01,0.01,0.01,0.01,0.79,0.19,0.01
30,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-20 06:00:00+00:00,12,105.0,,...,-5.0,False,NaN,0.01,0.01,0.01,0.13,0.69,0.15,0.01
31,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-20 18:00:00+00:00,24,95.0,,...,-15.0,False,NaN,0.01,0.01,0.13,0.45,0.28,0.08,0.01
32,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-21 06:00:00+00:00,36,80.0,,...,-30.0,False,NaN,0.01,0.14,0.44,0.24,0.09,0.04,0.01
33,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-21 18:00:00+00:00,48,70.0,,...,-40.0,False,NaN,0.02,0.31,0.41,0.13,0.05,0.03,0.01
34,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-22 06:00:00+00:00,60,60.0,,...,-50.0,False,NaN,0.08,0.59,0.21,0.06,0.02,0.01,0.01
35,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-22 18:00:00+00:00,72,50.0,,...,-60.0,False,NaN,0.23,0.50,0.15,0.05,0.02,0.01,0.01
36,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-23 18:00:00+00:00,96,30.0,INLAND,...,-80.0,False,NaN,0.55,0.31,0.05,0.01,0.01,0.01,0.01
37,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-24 18:00:00+00:00,120,NaN,DISSIPATED,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
df_ri_adj.loc[df_ri_adj['ATCF_ID'] == 'EP172023']

,ATCF_ID,BASIN,STORM_NUM,SEASON,NAME,INIT_TIME_UTC,VALID_TIME_UTC,TAU,VMAX_KT,STORM_TYPE,...,DELTA_VMAX_KT,RI,VMAX_ERROR,P(TD),P(TS),P(CAT1),P(CAT2),P(CAT3),P(CAT4),P(CAT5)
29,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-19 18:00:00+00:00,0,110.0,HURRICANE,...,0.0,False,0.0,0.0096,0.0096,0.0096,0.0096,0.7584,0.1824,0.0096
30,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-20 06:00:00+00:00,12,105.0,,...,-5.0,False,NaN,0.0096,0.0096,0.0096,0.1248,0.6624,0.1440,0.0096
31,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-20 18:00:00+00:00,24,95.0,,...,-15.0,False,NaN,0.0096,0.0096,0.1248,0.4320,0.2688,0.0768,0.0096
32,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-21 06:00:00+00:00,36,80.0,,...,-30.0,False,NaN,0.0096,0.1344,0.4224,0.2304,0.0864,0.0384,0.0096
33,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-21 18:00:00+00:00,48,70.0,,...,-40.0,False,NaN,0.0192,0.2976,0.3936,0.1248,0.0480,0.0288,0.0096
34,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-22 06:00:00+00:00,60,60.0,,...,-50.0,False,NaN,0.0768,0.5664,0.2016,0.0576,0.0192,0.0096,0.0096
35,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-22 18:00:00+00:00,72,50.0,,...,-60.0,False,NaN,0.2208,0.4800,0.1440,0.0480,0.0192,0.0096,0.0096
36,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-23 18:00:00+00:00,96,30.0,INLAND,...,-80.0,False,NaN,0.5500,0.3100,0.0500,0.0100,0.0100,0.0100,0.0100
37,EP172023,EP,17,2023,NORMA,2023-10-19 18:00:00+00:00,2023-10-24 18:00:00+00:00,120,NaN,DISSIPATED,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
df_ri_adj.loc[df_ri_adj['ATCF_ID'] == 'AL202023']

,ATCF_ID,BASIN,STORM_NUM,SEASON,NAME,INIT_TIME_UTC,VALID_TIME_UTC,TAU,VMAX_KT,STORM_TYPE,...,DELTA_VMAX_KT,RI,VMAX_ERROR,P(TD),P(TS),P(CAT1),P(CAT2),P(CAT3),P(CAT4),P(CAT5)
18,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-19 18:00:00+00:00,0,50.0,TROPICAL STORM,...,0.0,False,0.0,0.01,0.98,0.01,0.01,0.01,0.01,0.01
19,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-20 06:00:00+00:00,12,50.0,,...,0.0,False,NaN,0.02,0.90,0.06,0.01,0.01,0.01,0.01
20,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-20 18:00:00+00:00,24,55.0,,...,5.0,False,NaN,0.03,0.72,0.20,0.02,0.01,0.01,0.01
21,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-21 06:00:00+00:00,36,60.0,,...,10.0,False,NaN,0.03,0.55,0.32,0.05,0.02,0.01,0.01
22,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-21 18:00:00+00:00,48,65.0,,...,15.0,False,NaN,0.02,0.42,0.38,0.10,0.03,0.01,0.01
23,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-22 06:00:00+00:00,60,70.0,,...,20.0,False,NaN,0.01,0.28,0.52,0.12,0.03,0.01,0.01
24,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-22 18:00:00+00:00,72,75.0,,...,25.0,False,NaN,0.02,0.23,0.42,0.16,0.08,0.04,0.01
25,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-23 18:00:00+00:00,96,75.0,,...,25.0,False,NaN,0.02,0.21,0.41,0.18,0.08,0.04,0.01
26,AL202023,AL,20,2023,TAMMY,2023-10-19 18:00:00+00:00,2023-10-24 18:00:00+00:00,120,75.0,,...,25.0,False,NaN,0.03,0.20,0.38,0.17,0.10,0.04,0.02
